<!-- # Processing the final dataset
 -->


# Export data as CSV

In [9]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


In [10]:
field = ee.FeatureCollection(f'{data_folder}/field_biomass')

# select those with feature field_age greater than 22
# field = field.filter(ee.Filter.gt('field_age', 20))
# field = field.filter(ee.Filter.lt('field_biom', 70))

map = geemap.Map()
map.addLayer(field, {}, 'Field Biomass')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Main Model Dataset

### Age, Biomass

In [11]:
# Fire and Land Use
age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

ESA_CCI = ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51").select("AGB").rename("biomass")

fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
    .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
    .byte()
    .rename([str(year) for year in config.range_1985_2020])
    .gt(0)
    .reduce('sum').rename("num_fires")).unmask(0)

floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select(f"classification_{last_year}").eq(6)).rename("floodable_forests")


### Surrounding Landscape

In [12]:
quarters_ecoreg_biomass = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass")
distance_deep_forest = ee.Image(f"{data_folder}/distance_deep_forest").rename("dist")
sur_cover = ee.Image(f"{data_folder}/sur_cover")

distance_gt_1000 = distance_deep_forest.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
nearest_mature = ee.Image(f"{data_folder}/nearest_mature")

### Environmental

In [13]:
categorical = ee.Image(f"{data_folder}/categorical")

topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography") # 90m resolution

soil = ee.Image(f"{data_folder}/soilgrids")

terraclim = ee.Image(f"{data_folder}/terraclim_1958_2019")

In [14]:
# Load each scenario
historical = ee.Image(f"{data_folder}/CMIP6_historical")
ssp126 = ee.Image(f"{data_folder}/CMIP6_ssp126")
ssp245 = ee.Image(f"{data_folder}/CMIP6_ssp245")
ssp585 = ee.Image(f"{data_folder}/CMIP6_ssp585")

CMIP6_means = ee.Image(f"{data_folder}/CMIP6_means")

### Export CSV

- Total secondary forest area per 1km2
- Total pasture area per 1km2

In [15]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

pastureland = lulc.select("2020").eq(15).unmask(0).rename("pastureland")

# Aggregate the high-resolution pixels into the 1km grid
def reproject_export_image(image, name):
    image_area = image.reduceResolution(
        reducer = ee.Reducer.mean(),
        maxPixels = 1024,
        bestEffort = True # Use all pixels that can fit in the larger pixel
    ).reproject(
        crs = 'EPSG:4326',
        scale = 1000
    ).rename(name)

    export_image(image_area, name, region = roi, scale = 1000)


# reproject_export_image(pastureland, "pasture_area")
# reproject_export_image(age.gt(0).unmask(0), "secondary_area")

- 1 secondary forest pixel per 1km2
- 1 pasture pixel per 1km2

In [72]:
# create_grid(age.gt(0).unmask(0), "atlantic", cell_size = 1000, file_name = "secondary")
# create_grid(age.gt(0).unmask(0), "amazon", cell_size = 1000, file_name = "secondary")
# create_grid(pastureland, "amazon", cell_size = 1000, file_name = "pastureland")

# make one grid with all pixels that are NOT mature forests for nearest mature forest estimates
# non_mature_forests = mature_biomass_10k.eq(0).unmask(1).selfMask().rename("non_mature_forests")
# create_grid(non_mature_forests, "amazon", cell_size = 10000, file_name = "non_mature_forests")

In [38]:
soil.projection().nominalScale().getInfo()  # to initialize the projection

250

Since processing gets too heavy if we try to extract each point at a time for the entire Amazon at 1km resolution, we do it in chunks:

In [ ]:
# unified_data = ESA_CCI.addBands([age, 
#     fire, quarters_ecoreg_biomass,
#     nearest_mature, distance_deep_forest, sur_cover,
#     categorical, topography, soil,
#     ee.Image.pixelLonLat().rename(['lon', 'lat'])
# ])

# unified_data = ESA_CCI_reproj.addBands([age, nearest_mature])

unified_data = ESA_CCI #100m
nearest_mature #10k
unified_data = soil.addBands(terraclim.select(["mean_soil", "mean_vpd","mean_temp", 
                                                "mean_def", "mean_srad", "mean_pr",
                                                "mean_aet", "mean_pdsi"])) # 250



def export_csv(name, unified_data, grid_size, n_chunks = 10):
    
    if (name == "secondary"):
        secondary_area = ee.Image(f"{data_folder}/secondary_area")
        # unified_data = unified_data.addBands([age, floodable_forests, secondary_area,
        #                                       )

    elif (name == "pastureland"):
        pasture_area = ee.Image(f"{data_folder}/pasture_area")
        unified_data = unified_data.addBands([pasture_area, CMIP6_means])

    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_amazon_{name}")

    properties_to_export = unified_data.bandNames().getInfo()

    # 1. Get FeatureCollection size and calculate chunk size
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = unified_data.reduceRegions(
            collection = selected_pixels,
            reducer = ee.Reducer.first(),
            crsTransform = unified_data.projection().getInfo()['transform'],
            crs = unified_data.projection().getInfo()['crs']
        )

        # task = ee.batch.Export.table.toDrive(
        #     collection = unified_fc,
        #     description = f"grid_{grid_size}k_amazon_{name}_{chunk_index}",
        #     fileFormat = "CSV",
        #     selectors = [p for p in properties_to_export if p not in ['.geo', 'system:index']]
        # )
        # task.start()

        task = ee.batch.Export.table.toAsset(
            collection = unified_fc,
            description = f"grid_{grid_size}k_amazon_{name}_{chunk_index}",
            assetId = f"projects/amazon-forest-regrowth/assets/grid_{grid_size}k_amazon_{name}_ESA_CCI_100",
        )
        task.start()


    # for i in range(n_chunks):
    i = 0
    if i*chunk_size < total_features:
        process_chunk(i)

# export_csv("pastureland", unified_data, 1, n_chunks = 60)
export_csv("secondary", unified_data, 1, n_chunks = 25)
# export_csv("secondary", unified_data, 10, n_chunks = 15)

In [ ]:
grid_1k_reproj = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grid_1k_amazon_secondary_0")
grid_1k_ESA_CCI = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grid_1k_amazon_secondary_0_ESA_CCI")

ESA_CCI_reproj = ESA_CCI.reproject(age.projection())

map = geemap.Map(center=[-3.5, -60.24], zoom=6)
map.addLayer(ESA_CCI, {'min':50, 'max':200, 'palette':['blue', 'red']}, "ESA_CCI", opacity=0.4)
map.addLayer(ESA_CCI_reproj, {'min':50, 'max':200, 'palette':['blue', 'red']}, "ESA_CCI Reprojected", opacity=0.4)
# map.addLayer(age, {}, "Unified Data", opacity=0.05)
map.addLayer(grid_1k_reproj, {}, "Grid 1k")
# map.addLayer(grid_10k, {"color": "blue"}, "Grid 10k")
map

Map(center=[-3.5, -60.24], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [35]:
grid_1k_ESA_CCI = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grid_1k_amazon_secondary_ESA_CCI_100")
map.addLayer(grid_1k_ESA_CCI, {}, "Grid 1k ESA CCI 100m")


## Field Data

In [ ]:
field_data = ee.FeatureCollection(f"{data_folder}/field_biomass")

unified_fc = unified_data.reduceRegions(field_data, ee.Reducer.first(), 30)

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'unified_field',
    fileFormat = "CSV"
)
# task.start()

## Different land use aggregations


In [ ]:
suffixes = [
    "aggregated_all",
    "non_aggregated_all",
    "non_aggregated_10yr",
    "non_aggregated_5yr"
]

land_use_list = [
    ee.Image(f"{data_folder}/land_use_{suffix}")
    for suffix in suffixes
]

# Loop through each land use image
for land_use_image, suffix in zip(land_use_list, suffixes):

    selected_pixels = ee.FeatureCollection(f"{data_folder}/grid_10k_amazon_secondary")

    unified_fc = land_use_image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

    properties_to_export = land_use_image.bandNames().getInfo()

    task = ee.batch.Export.table.toDrive(
            collection = unified_fc,
            description = "unified_land_use_" + suffix,
            fileFormat = "CSV",
            selectors = properties_to_export
        )
    task.start()


In [34]:
selected_pixels = ee.FeatureCollection(f"{data_folder}/grid_10k_amazon_secondary")
# WITH BUFFER

mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k").rename("mature_biomass_10k")

selected_pixels = distance_deep_forest.reduceRegions(
    collection = selected_pixels,
    reducer = ee.Reducer.first(),
    scale = 10000
)

def buffer_feature(feature):
    distance = feature.getNumber('first').add(10000)
    buffer = feature.geometry().buffer(distance)
    return feature.setGeometry(buffer)

# Buffer each point to reach the nearest pixel
buffered_features = selected_pixels.map(buffer_feature)

# Extract the biomass value for each buffered region
# This will get the value from nearest valid pixel
nearest_mature = mature_biomass_10k.reduceRegions(
    collection=buffered_features,
    reducer=ee.Reducer.firstNonNull(),
    scale=10000
).map(lambda feature: feature.centroid())


unified_data = ESA_CCI.addBands(age)

unified_fc = unified_data.reduceRegions(nearest_mature, ee.Reducer.first(), 30)

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'buffered_nearest_mature',
    fileFormat = "CSV"
)
task.start()

# task = ee.batch.Export.table.toAsset(
#     collection = unified_fc,
#     description = 'unified_fc',
#     assetId = f"{data_folder}/unified_fc"
# )
# task.start()